In [1]:
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import StandardScaler

# Read CSV file and put it inside dataframe
df = pd.read_csv('train.csv')

# Show number of attributes
print(f'Number of attributes: {df.shape[1]}')

# Show names of attributes
print(f'Name of attributes: {df.columns}')

# Show number of instances
print(f'Number of instances: {df.shape[0]}')

# Show columns and top 5 rows
df.head()

Number of attributes: 12
Name of attributes: Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')
Number of instances: 891


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [2]:
# Check for duplicates after dropping PassengerId 
df = df.drop(columns='PassengerId')
print(f'Number of duplicated rows: {df.duplicated().sum()}')

# Show only columns with missing data
missing_data = df.isnull().sum()
print(f'{missing_data[missing_data > 0]}')

# Check columns that have more than 50% missing data
missing_proportion = df.isnull().mean()
columns_to_drop = missing_proportion[missing_proportion > 0.5]

# Drop those columns together with "Name" and "Ticket" columns
df = df.drop(columns=['Name','Ticket'] + list(columns_to_drop.index))


Number of duplicated rows: 0
Age         177
Cabin       687
Embarked      2
dtype: int64


In [3]:
# Create new 'Relatives' column based on number of relatives 
df['Relatives'] = df['Parch'] + df['SibSp']

df['Age'] = df['Age'].fillna(df['Age'].median())

# Drop SibSp, Parch columns
df = df.drop(columns=['SibSp','Parch'])
print(f'Number of attributes: {df.shape[1]}')
print(f'Name of attributes: {df.columns}')

Number of attributes: 7
Name of attributes: Index(['Survived', 'Pclass', 'Sex', 'Age', 'Fare', 'Embarked', 'Relatives'], dtype='object')


In [4]:
# Change Pclass into category datatype
df['Pclass'] = df['Pclass'].astype('category')
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   Survived   891 non-null    int64   
 1   Pclass     891 non-null    category
 2   Sex        891 non-null    object  
 3   Age        891 non-null    float64 
 4   Fare       891 non-null    float64 
 5   Embarked   889 non-null    object  
 6   Relatives  891 non-null    int64   
dtypes: category(1), float64(2), int64(2), object(2)
memory usage: 42.9+ KB


,Survived,Pclass,Sex,Age,Fare,Embarked,Relatives
0,0,3,male,22.0,7.2500,S,1
1,1,1,female,38.0,71.2833,C,1
2,1,3,female,26.0,7.9250,S,0
3,1,1,female,35.0,53.1000,S,1
4,0,3,male,35.0,8.0500,S,0


In [5]:
# Do one-hot encoding
data_encoded = pd.get_dummies(df,drop_first = True)
print(data_encoded)

     Survived   Age     Fare  Relatives  Pclass_2  Pclass_3  Sex_male  \
0           0  22.0   7.2500          1     False      True      True   
1           1  38.0  71.2833          1     False     False     False   
2           1  26.0   7.9250          0     False      True     False   
3           1  35.0  53.1000          1     False     False     False   
4           0  35.0   8.0500          0     False      True      True   
..        ...   ...      ...        ...       ...       ...       ...   
886         0  27.0  13.0000          0      True     False      True   
887         1  19.0  30.0000          0     False     False     False   
888         0  28.0  23.4500          3     False      True     False   
889         1  26.0  30.0000          0     False     False      True   
890         0  32.0   7.7500          0     False      True      True   

     Embarked_Q  Embarked_S  
0         False        True  
1         False       False  
2         False        True  
3  

In [6]:
# Split data into training and testing data
label = data_encoded['Survived']
attributes = data_encoded.drop(columns='Survived')

attributes_train, attributes_test, label_train, label_test = train_test_split(attributes, label, test_size=0.3)

# Scale the training data 
scaler = StandardScaler()
attributes_train = scaler.fit_transform(attributes_train)
attributes_test = scaler.transform(attributes_test)

In [7]:
# Initialize the k-NN classifier with a chosen k value (e.g., k=5)
knn = KNeighborsClassifier(n_neighbors=5)

# Train the k-NN model on the training data
knn.fit(attributes_train, label_train)

# Predict on the test set
label_pred = knn.predict(attributes_test)

# Show accuracy and confusion matrix for testing set
accuracy = accuracy_score(label_test, label_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

ConfusionMatrix = confusion_matrix(label_test,label_pred)
print(ConfusionMatrix)

Accuracy: 79.85%
[[140  23]
 [ 31  74]]
